# Microsoft Fabric API - Authentication with Service Principal

This notebook demonstrates how to use the `microsoft-fabric-api` package to authenticate and interact with Microsoft Fabric using a Service Principal.

## Step 1: Install the Package

In [ ]:
%pip install microsoft-fabric-api azure-identity

## Step 2: Import Required Libraries

In [ ]:
import os
from azure.identity import ClientSecretCredential
from microsoft_fabric_api import FabricClient

## Step 3: Configure Service Principal Credentials

Set your Azure Service Principal credentials. These should match the GitHub secrets used in your CI/CD pipeline:
- `AZURE_CLIENT_ID`: Application (client) ID
- `AZURE_CLIENT_SECRET`: Client secret value
- `AZURE_TENANT_ID`: Directory (tenant) ID

**Security Note**: Never commit credentials to source control. Use environment variables or secure secret management.

In [ ]:
# Option 1: Load from environment variables (recommended)
client_id = os.getenv("AZURE_CLIENT_ID", "your-client-id-here")
client_secret = os.getenv("AZURE_CLIENT_SECRET", "your-client-secret-here")
tenant_id = os.getenv("AZURE_TENANT_ID", "your-tenant-id-here")

# Option 2: Directly set values (for testing only - DO NOT commit)
# client_id = "your-client-id-here"
# client_secret = "your-client-secret-here"
# tenant_id = "your-tenant-id-here"

print(f"Client ID configured: {client_id[:8]}..." if len(client_id) > 8 else "Not configured")
print(f"Tenant ID configured: {tenant_id[:8]}..." if len(tenant_id) > 8 else "Not configured")

## Step 4: Authenticate with Service Principal

In [ ]:
# Create Azure credential using Service Principal
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

# Initialize Fabric API client
fabric_client = FabricClient(token_credential=credential)

print("✓ Successfully authenticated with Microsoft Fabric API")

## Step 5: Test Authentication - List Workspaces

Test the connection by listing all workspaces the Service Principal has access to.

In [ ]:
# List all workspaces
workspaces = list(fabric_client.core.workspaces.list_workspaces())

print(f"Found {len(workspaces)} workspace(s):\n")
for ws in workspaces:
    print(f"  • {ws.display_name} (ID: {ws.id})")

## Example: Get Workspace Items

Retrieve all items from a specific workspace.

In [ ]:
# Replace with your workspace name or ID
workspace_name = "[D] Fabric Blueprint"  # Example from config.yml

# Find workspace by name
target_workspace = None
for ws in workspaces:
    if ws.display_name == workspace_name:
        target_workspace = ws
        break

if target_workspace:
    # Get items in workspace
    items = list(fabric_client.core.items.list_items(workspace_id=target_workspace.id))
    
    print(f"\nItems in workspace '{workspace_name}':\n")
    for item in items:
        print(f"  • {item.display_name} ({item.type})")
else:
    print(f"Workspace '{workspace_name}' not found.")

## API Discovery

Since the documentation is limited, let's explore the API structure programmatically.

In [ ]:
# Explore the FabricClient structure
print("FabricClient attributes:")
print("=" * 60)
for attr in dir(fabric_client):
    if not attr.startswith('_'):
        obj = getattr(fabric_client, attr)
        print(f"  • {attr}: {type(obj).__name__}")

In [ ]:
# Explore the 'core' module methods
if hasattr(fabric_client, 'core'):
    print("\nfabric_client.core attributes:")
    print("=" * 60)
    for attr in dir(fabric_client.core):
        if not attr.startswith('_'):
            obj = getattr(fabric_client.core, attr)
            print(f"  • core.{attr}: {type(obj).__name__}")

In [ ]:
# Explore workspaces operations
if hasattr(fabric_client.core, 'workspaces'):
    print("\nfabric_client.core.workspaces methods:")
    print("=" * 60)
    for attr in dir(fabric_client.core.workspaces):
        if not attr.startswith('_') and callable(getattr(fabric_client.core.workspaces, attr)):
            print(f"  • workspaces.{attr}()")

In [ ]:
# Explore items operations
if hasattr(fabric_client.core, 'items'):
    print("\nfabric_client.core.items methods:")
    print("=" * 60)
    for attr in dir(fabric_client.core.items):
        if not attr.startswith('_') and callable(getattr(fabric_client.core.items, attr)):
            print(f"  • items.{attr}()")

In [ ]:
# Get help documentation for a specific method
import inspect

# Example: Get signature and docstring for list_workspaces
method = fabric_client.core.workspaces.list_workspaces
print("\nMethod: list_workspaces")
print("=" * 60)
print(f"Signature: {inspect.signature(method)}")
print(f"\nDocstring:\n{inspect.getdoc(method)}")

In [ ]:
# Get help for list_items method
method = fabric_client.core.items.list_items
print("\nMethod: list_items")
print("=" * 60)
print(f"Signature: {inspect.signature(method)}")
print(f"\nDocstring:\n{inspect.getdoc(method)}")

## Documentation Resources

Based on the PyPI page, here are the key resources:

1. **Microsoft Fabric REST API documentation**: https://learn.microsoft.com/en-us/rest/api/fabric
2. **Package Version**: 0.1.0b2 (beta release - still evolving)
3. **Authentication Guide**: https://learn.microsoft.com/en-us/rest/api/fabric/articles/get-started/fabric-api-quickstart

**Key Pattern**: The SDK follows the REST API structure:
- `fabric_client.core.*` - Maps to Core API endpoints
- Methods generally return `ItemPaged` objects (convert to `list()` for iteration)
- Uses `TokenCredential` from `azure-identity` for authentication

Since this is a beta package wrapping the REST API, you can reference the REST API docs to understand available operations.